<p align="center">
  <img width='275' src="https://user-images.githubusercontent.com/8030363/195494428-aef60f6c-5211-43ac-8650-ddaa05bfa34a.png" />
</p>

***
***

# Tutorial: Knowledge Graph Entity Search

***

**Author:** [TJCallahan](http://tiffanycallahan.com/)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Wiki Page:** [Drug Safety Tutorial](https://github.com/callahantiff/PheKnowLator/wiki/KG-based-Investigation-of-Drug-Outcome-Pairs)  
<!-- **Tutorial Slides:** [PheKnowLator_Tutorial_EntitySearch](https://docs.google.com/presentation/d/19oc41CFILLlf0HjJfFJlUER-4wt1YV-BItkqWbI4NwU/edit?usp=sharing)  
**Release:** **[v3.0.2](https://github.com/callahantiff/PheKnowLator/wiki/v3.0.0)**   -->
  
<br> 

## Purpose  
The goal of this notebook is to explore different ways to examine relationships between different types of entities in a `PheKnowLator` knowledge graph. A figure depicting the general workflow that is used in this demo is shown below. The objective is to search `OMOP` (Observational Medical Outcomes Partnership) common data model (CDM) concepts against entities in a `PheKnowLator` knowledge graph, which requires mappings from `OMOP2OBO`. Each of these resources will be introduced and described in this tutorial.

<br>

![](https://user-images.githubusercontent.com/8030363/195580089-872994c4-6498-491c-90e6-51d4203aec93.png)

<br>

In this tutorial, we will demonstrate the utility of this workflow by examining three different types of adverse event relationships will be explored:  

**Known Relationships**  
- **Positive:** *Known drug-outcome pairs:*  
  - `lisinopril` ([OMOP:1308216](https://athena.ohdsi.org/search-terms/terms/1308216)) and `myocardial infarction` ([OMOP:4329847](https://athena.ohdsi.org/search-terms/terms/4329847)) 
  
- **Negative:** *Drug-outcome pairs known to not be related:*   
  - `lisinopril` ([OMOP:1308216](https://athena.ohdsi.org/search-terms/terms/1308216)) and `contact dermatitis` ([OMOP:134438](https://athena.ohdsi.org/search-terms/terms/134438)) 
  - `lisinopril` ([OMOP:1308216](https://athena.ohdsi.org/search-terms/terms/1308216)) and `ingrown toenails` ([OMOP:4290993](https://athena.ohdsi.org/search-terms/terms/4290993)) 
  - `lisinopril` ([OMOP:1308216](https://athena.ohdsi.org/search-terms/terms/1308216)) and `presbyopia` ([OMOP:373478](https://athena.ohdsi.org/search-terms/terms/373478)) 

**Unknown Relationships**  
*Drug-outcome pairs with no known relationship:*   
- `ivermectin` ([OMOP:42593521](https://athena.ohdsi.org/search-terms/terms/42593521)) and `neurotoxicity` ([OMOP:4024301](https://athena.ohdsi.org/search-terms/terms/4024301)) 

<br>

___

### Notebook Organization  
- [Set-Up Environment](#set-environment)  
- [Knowledge Graph Data](#kg-data)   

**Prerequisites**  
- [OMOP2OBO Mappings](#omop2obo-mappings)   
- [PheKnowLator Knowledge Graph - Schema](#pkt-schema)  

**Drug Safety Task - Knowledge Graph-based Drug Safety**   
- [Knowledge-based Characterization](#kg-characterization)  
    - [Node-Level](#node-level)  
    - [Path-Level](#path-level) 

***
***

<br>

<br>

***  
## Set-Up Environment  <a class="anchor" id="set-environment"></a> 
***  
___
  

In [ ]:
# # uncomment and run to install any required modules from notebooks/requirements.txt
# import sys
# !{sys.executable} -m pip install -r ../../notebooks/requirements.txt

In [ ]:
# run only if running a local version (i.e., forked from GitHub) of pkt_kg
import sys
sys.path.append('../../../')

⚠️ **Installing graphviz** ⚠️  
graphviz can be a tricky library to install, which is why it is not automatically handled for the user via `notebooks/requirements.txt`. If you want to install this, I recommend using the last comment in the following [StackOverflow](https://stackoverflow.com/questions/69377104/unable-to-execute-dot-command-after-installation-of-graphviz-there-is-no-layou) post, but make sure you understand what it is doing before running any of the code. The only thing this impacts is the rendering of network visualizations. Simply skip the visualization code chunks if you are unable to install `graphviz`.

In [ ]:
# import needed libraries
import glob
import json
import networkx as nx
import os
import pandas as pd
import pickle
import random
import re

from entity_search import *
from pyvis.network import Network
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDFS
from tqdm.notebook import tqdm
from typing import Callable, Dict, List, Optional, Union

# temp work around to avoid logging error
try: from pkt_kg.utils import *
except FileNotFoundError: from pkt_kg.utils import *

# create namespace for OBO ontologies
obo = Namespace('http://purl.obolibrary.org/obo/')

<br>

***

## Knowledge Graph Data  <a class="anchor" id="kg-data"></a>
***

#### *How to access PheKnowLator Data*
___

This notebook was built using a `v3.0.2` OWL-NETS-abstracted subclass-based build with inverse relations, which is publicly available and can be downloaded using the following links:  
- [PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle](https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NetworkxMultiDiGraph.gpickle)  
- [PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NodeLabels.txt](https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/inverse_relations/owlnets/PheKnowLator_v3.0.2_full_subclass_inverseRelations_OWLNETS_NodeLabels.txt)  


### Download Data  
***

The knowledge graph data is publicly available and downloaded from the PheKnowLator project's Google Cloud Storage Bucket: https://console.cloud.google.com/storage/browser/pheknowlator/. Data will be downloaded to the `tutorials/entity_search/data` directory.

In [ ]:
# notebook will create a temporary directory and will download data to it
write_location = 'data/'
if not os.path.exists(write_location): os.mkdir(write_location)

In [ ]:
# download data to the data directory
data_urls = [
    'https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/relations_only/owlnets/PheKnowLator_v3.0.2_full_subclass_relationsOnly_OWLNETS_NetworkxMultiDiGraph.gpickle',
    'https://storage.googleapis.com/pheknowlator/current_build/knowledge_graphs/subclass_builds/relations_only/owlnets/PheKnowLator_v3.0.2_full_subclass_relationsOnly_OWLNETS_NodeLabels.txt'
]

for url in data_urls:
    file_name = url.split('/')[-1] if 'metadata' not in url else re.sub(r'\?.*', '', url.split('/')[-1])
    if not os.path.exists(write_location + file_name): data_downloader(url, write_location, file_name)
        

### Load Data
***

The knowledge graph will be loaded as a `networkx` MultiDiGraph object and the node labels will be read in and converted to a dictionary to enable easy access to node labels and other relevant metadata.

#### Knowledge Graph

Note that this file is large and can take up to 2 minutes to load on a laptop with 16GB of Ram.

In [ ]:
# load the knowledge graph
with open(write_location + data_urls[0].split('/')[-1], 'rb') as f:
    kg = pickle.load(f)

undirected_kg = nx.to_undirected(kg)
print('The knowledge graph contains {} nodes and {} edges'.format(len(kg.nodes()), len(kg.edges())))

#### Load Node Metadata

In [ ]:
# read in node metadata
data_loc = write_location + data_urls[1].split('/')[-1]
node_data = pd.read_csv(data_loc, header=0, sep=r"\t", encoding="utf8", engine='python', quoting=3)
node_data['entity_uri'] = node_data['entity_uri'].str.strip('<>')  # remove angle brackets
node_data.head()

In [ ]:
# convert node data to dictionary
node_data_dict = dict()
for idx, row in tqdm(node_data.iterrows(), total=node_data.shape[0]):
    node_data_dict[row['entity_uri']] = {
        'label': row['label'],
        'description': row['description/definition']
    }

<br>

***
***

# Prerequisites 


<br>

***

## OMOP2OBO Mappings <a class="anchor" id="omop2obo-mappings"></a> <img width="225" align="right" src="https://user-images.githubusercontent.com/8030363/195580079-06726eac-38e8-44c9-8de6-32179bca9614.png">
***
____


**GitHub:** https://github.com/callahantiff/OMOP2OBO  
**Dashboard:**  http://tiffanycallahan.com/OMOP2OBO_Dashboard/  
**Zenodo Mapping Archive:** [Condition Occurrence](https://zenodo.org/record/6949688); [Drug Exposure Ingredients](https://zenodo.org/record/6949696); [Measurement](https://zenodo.org/record/6949858)  

<br>

<a target="_blank" href="https://user-images.githubusercontent.com/8030363/195500004-0e20019c-f33b-40a6-b3be-739b1136021a.png"> <img src="https://user-images.githubusercontent.com/8030363/195500004-0e20019c-f33b-40a6-b3be-739b1136021a.png" width="600"></a> 

<br>

This tutorial depends on mappings from the [`OMOP2OBO`](https://github.com/callahantiff/OMOP2OBO/wiki) in order to align concepts in the [OMOP (Observational Medical Outcomes Partnership) Common Data Model](https://ohdsi.github.io/CommonDataModel/cdm60.html#NOTE_ABOUT_CDM_v60). An example of the mappings is shown in the figure above. As shown in this figure, mappings were created to eight Open Biological and Biomedical Ontology (OBO) Foundry ontologies spanning diseases (Mondo Disease Ontology [`Mondo`]), phenotypes (Human Phenotype Ontology [`HPO`]), anatomical entities (Uber Anatomy Ontology [`Uberon`]; Cell Ontology [`CL`]), organisms (National Center for Biotechnology Information Taxon Ontology [`NCBITaxon`]), chemicals (Chemical Entities of Biological Interest [`ChEBI`]), vaccines (the Vaccine Ontology [`VO`]), and proteins (the Protein Ontology [`PRO`]).

A script ([`process_omop2obo_mappings.py`](https://github.com/callahantiff/PheKnowLator/tree/master/notebooks/tutorials/entity_search/process_omop2obo_mappings.py)) has been developed to download the latest OMOP2OBO mappings for the OMOP [Condition Occurrence](https://ohdsi.github.io/CommonDataModel/cdm60.html#CONDITION_OCCURRENCE) and [Drug Exposure ingredients](https://ohdsi.github.io/CommonDataModel/cdm60.html#DRUG_EXPOSURE) tables (please verify this when running this Notebook using the links provided above). Once downloaded, the mappings are converted to a dictionary and pickled to a file called `notebooks/tutorials/entity_search/data/omop2obo_mapping_dict.pkl`. The code chunk below runs this script and reads in the resulting dictionary.

**Mapping Example:** 
The following example demonstrates how to use the mapping dictionary. In this example, the `OMOP` concept [`256439`](https://athena.ohdsi.org/search-terms/terms/256439) is searched and returns mappings to both `Mondo` and `HPO`.

The structure of the dictionary is shown below: 

```python
omop2obo_mappings[256439]

{
    'concept_name': 'Allergic rhinitis due to pollen',
    'mapping(s)': {
        'HP': {
            'mapping_logic': 'OR',
            'id': 'HP_0012395 | HP_0003193',
            'label': 'seasonal allergy | allergic rhinitis'
        },
        'MONDO': {
            'mapping_logic': nan,
            'id': 'MONDO_0011786',
            'label': 'allergic rhinitis'
        }
    }
}
```

<br>


In [ ]:
# run script to download and process OMOP2OBO mappings
%run process_omop2obo_mappings.py

In [ ]:
# load the mapping dictionary
omop2obo_mappings = pickle.load(open(write_location + 'omop2obo_mapping_dict.pkl', 'rb'))

<br>

***

## PheKnowLator Knowledge Graph Schema <a class="anchor" id="pkt-schema"></a>
***
____

The interactive figures below (go ahead, try hovering over the nodes and edges, it's awesome ) provide a high-level overview of the current (`v3.0.2`) PheKnowLator knowledge graph schema (i.e., the primary node types and relationships). Note that there are two figures shown below.
1. The first figure illustrates the relationships between the core set of 12 OBO Foundry ontologies when including their imported ontologies.
2. The second figure Illustrates the edges that are added to the core set of merged ontologies.

Shared colors between the figures represent a shared resource. For example, chemicals, cofactors, and catalysts share the same color (maroon) and are part of ChEBI. This is the same for the RO, which is represented in the first figure as the black lines between nodes. The green and yellow rectangles indicate data sources that are not from an OBO Foundry ontology and the specific ontology used to integrate them with the core set of ontologies in the first figure. For example, variant, transcript, and gene data are connected to the core ontology set via the SO.

Acronyms: `CL` (Cell ontology); `CLO` (Cell Line Ontology); `ChEBI` (Chemical Entities of Biological Interest); `GO` (Gene Ontology); `HPO` (Human Phenotype Ontology); `Mondo` (Mondo Disease Ontology); `PRO` (Protein Ontology); `PW` (Pathway Ontology); `SO` (Sequence Ontology); `VO` (Vaccine Ontology); `Uberon` (Uber-Anatomy Ontology).


For more information please see the associated Wiki page (https://github.com/callahantiff/PheKnowLator/wiki).

In [ ]:
# visualize the core set of OBOs used in phenknowlator knowledge graphs
g = visualize_pheknowlator_ontologies()
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=3)   
g.toggle_physics(True)
g.show('PheKnowLator_OBOs.html')


In [ ]:
# visualize edges added to the core set of OBO ontologies
g = visualize_pheknowlator_schema()
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=3)   
g.toggle_physics(True)
g.show('PheKnowLator_Schema.html')

<br>

***
***

# Drug Safety Task - Knowledge Graph-based Drug Safety 

<br>

<br>

***

## Knowledge-based Characterization  <a class="anchor" id="kg-characterization"></a>
***
____

The goal is to use the knowledge graph to explore what we know about specific concepts as well as what we can say about pairs of concepts. Additional details are presented by comparison below:

#### [Node-Level](#node-level)
 - <u>Node Ancestry</u>: Identify all ancestors for each node up to the root.
 - <u>Node Neighborhood</u>: Returns all nodes reachable from a node of interest via a single directed edge.   


#### [Path-Level](#path-level)
  - <u>Shortest Path</u>: Returns the single shortest path.  
  - <u>Simple Paths</u>: Returns all paths with no repeated nodes of varying length.  
  - <u>Efficiency</u>: A measure of how efficiently information is exchanged between nodes, ranging from 0-1, where 1 indicates high efficiency.

---

### Look-up PheKnowLator Entities  
use the `OMOP2OBO` mapping dictionary to obtain PheKnowLator entities for the following `OMOP` concepts:  
- lisinopril [`1308216`](https://athena.ohdsi.org/search-terms/terms/1308216)  
- ivermectin [`1784444`](https://athena.ohdsi.org/search-terms/terms/1784444)  
- Myocardial Infarction [`4329847`](https://athena.ohdsi.org/search-terms/terms/4329847)  
- Contact Dermatitis [`134438`](https://athena.ohdsi.org/search-terms/terms/134438)  
- Ingrowing Toenail [`4290993`](https://athena.ohdsi.org/search-terms/terms/4290993)  
- Presbyopia [`373478`](https://athena.ohdsi.org/search-terms/terms/373478)  
- neurotoxicity [`4024301`](https://athena.ohdsi.org/search-terms/terms/4024301)  

<br>

#### 💡 Concept Search  
- [Athena](https://athena.ohdsi.org/) is a great tool to use when looking looking for the OMOP concepts  
- [Ontology Lookup Service](https://www.ebi.ac.uk/ols/) is a very useful tool for  

<br>


In [ ]:
# lisinopril
omop2obo_mappings[1308216]

In [ ]:
# ivermectin
omop2obo_mappings[1784444]

In [ ]:
# myocardial infarction
omop2obo_mappings[4329847]

In [ ]:
# contact dermatits
omop2obo_mappings[134438]

In [ ]:
# ingrowing toenails
omop2obo_mappings[4290993]

In [ ]:
# presbyopia
omop2obo_mappings[373478]

In [ ]:
# neurotoxicity
omop2obo_mappings[4024301]

<br>

### Node-Level  <a class="anchor" id="node-level"></a>
***

#### Nodes  
- **Drugs**
  - [lisinopril (`CHEBI_6503`)](#chebi_6503)  
  - [ivermectin (`CHEBI_6078`)](#chebi_6078)  
- **Outcomes (diseases)**
  - [myocardial infarction (`MONDO_0005068`)](#mondo_0005068)  
  - [contact dermatitis (`MONDO_0005480`)](#mondo_0005480)    
  - [ingrowing toenail (`HP_0012710`)](#hp_0012710)  
  - [presbyopia (`MONDO_0001330`)](#mondo_0001330)  
  - [neurotoxicity (`MONDO_0005527`)](#mondo_0005527)  

***

#### lisinopril (`CHEBI_6503`) <a class="anchor" id="chebi_6503"></a>

In [ ]:
node = [obo.CHEBI_6503]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'CHEBI'
chebi6503_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
chebi6503_ancestors = format_path_ancestors(chebi6503_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(chebi6503_ancestors)  # using static viz
g = visualize_kg_output(kg, chebi6503_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))


In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(chebi6503_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in chebi6503_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))
        

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
chebi6503_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, chebi6503_sorted_neigbors, node_data_dict, verbose=False)

***

#### ivermectin (`CHEBI_6078`) <a class="anchor" id="chebi_6078"></a>

In [ ]:
node = [obo.CHEBI_6078]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'CHEBI'
chebi6078_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
chebi6078_ancestors = format_path_ancestors(chebi6078_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(chebi6078_ancestors)  # using static viz
g = visualize_kg_output(kg, chebi6078_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))


In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(chebi6078_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in chebi6078_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
chebi6078_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, chebi6078_sorted_neigbors, node_data_dict, verbose=False)

***
***

#### myocardial infarction (`MONDO_0005068`) <a class="anchor" id="mondo_0005068"></a>

In [ ]:
node = [obo.MONDO_0005068]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'MONDO'
mondo0005068_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
mondo0005068_ancestors = format_path_ancestors(mondo0005068_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(mondo0005068_ancestors)  # using static viz
g = visualize_kg_output(kg, mondo0005068_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))

In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(mondo0005068_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in mondo0005068_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
mondo0005068_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, mondo0005068_sorted_neigbors, node_data_dict, verbose=False)


***

#### contact dermatitis (`MONDO_0005480`) <a class="anchor" id="mondo_0005480"></a>

In [ ]:
node = [obo.MONDO_0005480]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'MONDO'
mondo0005480_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
mondo0005480_ancestors = format_path_ancestors(mondo0005480_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(mondo0005480_ancestors)  # using static viz
g = visualize_kg_output(kg, mondo0005480_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))

In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(mondo0005480_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in mondo0005480_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
mondo0005480_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, mondo0005480_sorted_neigbors, node_data_dict, verbose=False)

***

#### Ingrowing Toenail (`HP_0012710`) <a class="anchor" id="hp_0012710"></a>

In [ ]:
node = [obo.HP_0012710]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'HP'
hp0012710_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
hp0012710_ancestors = format_path_ancestors(hp0012710_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(hp0012710_ancestors)  # using static viz
g = visualize_kg_output(kg, hp0012710_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))


In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(hp0012710_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in hp0012710_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
hp0012710_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, hp0012710_sorted_neigbors, node_data_dict, verbose=False)

***

#### Presbyopia (`MONDO_0001330`) <a class="anchor" id="mondo_0001330"></a>

In [ ]:
node = [obo.MONDO_0001330]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'MONDO'
mondo0001330_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
mondo0001330_ancestors = format_path_ancestors(mondo0001330_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(mondo0001330)  # using static viz
g = visualize_kg_output(kg, mondo0001330_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))


In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(mondo0001330_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in mondo0001330_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
mondo0001330_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, mondo0001330_sorted_neigbors, node_data_dict, verbose=False)

***

#### Neurotoxicity (`MONDO_0005527`) <a class="anchor" id="mondo_0005527"></a>

In [ ]:
node = [obo.MONDO_0005527]
degree = str(kg.degree(node[0]))
in_edges = str(len(kg.in_edges(node[0])))
out_edges = str(len(kg.out_edges(node[0])))
print('This node has degree {}, which consists of {} in-edges and {} out-edges'.format(degree, in_edges, out_edges))

*Ancestors*

In [ ]:
# examine the node's ancestors
prefix = 'MONDO'
mondo0005527_anc_dict = processes_ancestor_path_list(nx_ancestor_search(kg, node.copy(), prefix))
mondo0005527_ancestors = format_path_ancestors(mondo0005527_anc_dict, node_data_dict)

In [ ]:
# visualize hierarchy
# visualize_ancestor_tree(mondo0005527_ancestors)  # using static viz
g = visualize_kg_output(kg, mondo0005527_ancestors, node_data_dict)
g.force_atlas_2based(gravity=-25, central_gravity=0.001, spring_length=90, spring_strength=0.08, damping=2, overlap=0)   
g.toggle_physics(True)
g.show('{}_Ancestor_Graph.html'.format(str(node[0]).split('/')[-1]))


In [ ]:
# print results -- nodes are ordered by seniority (higher numbers indicate closer to root)
print('Ancestors of {}'.format(node[0]))
if node_data_dict[str(node[0])]['description'] != 'None': print('- Definition: {}\n'.format(node_data_dict[str(node[0])]['description']))
for level in range(len(mondo0005527_ancestors)):
    print('Level: {}'.format(str(level)))
    for v in mondo0005527_ancestors[::-1][level]:
        v_uri = URIRef(v.split('(')[-1].strip(')'))
        descs = len([a[1] for b in [[[i, n[0]] for j in [kg.get_edge_data(*(n[0], v_uri)).keys()]
                                     for i in j] for n in list(kg.in_edges(v_uri))] for a in b
                     if a[0] == RDFS.subClassOf])
        uri_strip = re.sub('http://purl.obolibrary.org/obo/', '', v)
        print('- {}'.format(uri_strip))
        print('- Definition: {}'.format(node_data_dict[str(v_uri)]['description']))
        print('- Descendants: {}\n'.format(descs + 1))

*Neighborhood*

In [ ]:
# examine the node's neigborhood (out-edges)
nodes = list(kg.neighbors(node[0]))
neighbors = [a for b in [[[i, n] for j in [kg.get_edge_data(*(node[0], n)).keys()]
                          for i in j] for n in nodes] for a in b]
mondo0005527_sorted_neigbors = sorted(neighbors, key=lambda x: (str(x[1]).split('/')[-1], x[0]))

print('{} has {} neighbors'.format(str(node[0]).split('/')[-1], degree))

In [ ]:
# print node's neigborhood (out-edges)
formats_node_information(node, mondo0005527_sorted_neigbors, node_data_dict, verbose=False)

<br>

### Path-Level Characterization  <a class="anchor" id="path-level"></a>

***

- **Positive Relationship:**  
  - [`lisinopril` ([CHEBI_6503](http://purl.obolibrary.org/obo/CHEBI_6503)) and `myocardial infarction` ([MONDO_0005068](http://purl.obolibrary.org/obo/MONDO_0005068))](#path1)  
   
- **Negative Relationships:**  
  - [`lisinopril` ([CHEBI_6503](http://purl.obolibrary.org/obo/CHEBI_6503)) and `contact dermatitis` ([MONDO_0005480](http://purl.obolibrary.org/obo/MONDO_0005480))](#path2)       
  - [`lisinopril` ([CHEBI_6503](http://purl.obolibrary.org/obo/CHEBI_6503)) and `ingrowing toenails` ([HP_0012710](http://purl.obolibrary.org/obo/MONDO_0005480))](#path3)       
  - [`lisinopril` ([CHEBI_6503](http://purl.obolibrary.org/obo/CHEBI_6503)) and `presbyopia` ([MONDO_0001330](http://purl.obolibrary.org/obo/MONDO_0001330))](#path4)       

- **Unknown Relationships:**  
  - [`ivermectin` ([CHEBI_6078](http://purl.obolibrary.org/obo/CHEBI_6078)) and `neurotoxicity` ([MONDO_0005527](http://purl.obolibrary.org/obo/MONDO_0005527))](#path5)   


***

### Positive Relationships

When examining `lisinopril` and `myocardial infarction`. We observed that this pair had an efficiency of `1.0` and a single shortest path of length `1`.

***

#### lisinopril (`CHEBI_6503`) and myocardial infarction (`MONDO_0005068`) <a class="anchor" id="path1"></a> 

In [ ]:
s = obo.CHEBI_6503; t = obo.MONDO_0005068
try:
    kg_orientation = kg; ind = 'directed'
    spl_d = nx.shortest_path_length(kg, source=s, target=t)  
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))
    # get efficiency (measure of how efficiently information is exchanged between nodes)
    eff = nx.efficiency(undirected_kg, s, t)
    print('The efficiency between the nodes is: {}'.format(str(eff)))
except nx.NetworkXNoPath:
    print('There is no path between {} and {}'.format(str(s), str(t)))
    kg_orientation = undirected_kg; ind = 'undirected'
    spl_d = nx.shortest_path_length(undirected_kg, source=s, target=t)
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))

In [ ]:
shortest_paths1 = list(nx.all_shortest_paths(kg_orientation, s, t))
p_len = nx.shortest_path_length(kg_orientation, source=s, target=t)
v = 'is' if len(shortest_paths1) == 1 else 'are'
print('There {} {} shortest paths of length {}'.format(v, str(len(shortest_paths1)), str(p_len)))

In [ ]:
formats_path_information(kg=kg_orientation,
                         paths=shortest_paths1,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

*Simple Paths*

In [ ]:
# simple paths (i.e., no repeated nodes)
simple_paths1 = []; counter = 0
for path in tqdm(nx.all_simple_paths(kg_orientation, source=s, target=t, cutoff=5)):
    simple_paths1 += [path]
    if counter == 25: break
    else: counter += 1

In [ ]:
# print random sample of n simple paths
formats_path_information(kg=kg_orientation,
                         paths=simple_paths1,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

<br>

***

### Negative Relationships  

When examining the following negative relationships:  
1. `lisinopril` and `contact dermatitis`. This pair had an efficiency of `0.33` and `14` shortest paths of length 4.  
2. `lisinopril` and `ingrowing toenails`. This pair had an efficiency of `0.33` and no shortest paths.
3.  `lisinopril` and `presbyopia`. This pair had an efficiency of `0.25` and no shortest paths.


***
#### lisinopril (`CHEBI_6503`) and contact dermatitis (`MONDO_0005480`)  <a class="anchor" id="path2"></a>


In [ ]:
s = obo.CHEBI_6503; t = obo.MONDO_0005480
try:
    kg_orientation = kg; ind = 'directed'
    spl_d = nx.shortest_path_length(kg, source=s, target=t)  
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))
    # get efficiency (measure of how efficiently information is exchanged between nodes)
    eff = nx.efficiency(undirected_kg, s, t)
    print('The efficiency between the nodes is: {}'.format(str(eff)))
except nx.NetworkXNoPath:
    print('There is no path between {} and {}'.format(str(s), str(t)))
    kg_orientation = undirected_kg; ind = 'undirected'
    spl_d = nx.shortest_path_length(undirected_kg, source=s, target=t)
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))

In [ ]:
shortest_paths2 = list(nx.all_shortest_paths(kg_orientation, s, t))
p_len = nx.shortest_path_length(kg_orientation, source=s, target=t)
v = 'is' if len(shortest_paths2) == 1 else 'are'
print('There {} {} shortest paths of length {}'.format(v, str(len(shortest_paths2)), str(p_len)))

In [ ]:
formats_path_information(kg=kg_orientation,
                         paths=shortest_paths2,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

*Simple Paths*

In [ ]:
# simple paths (i.e., no repeated nodes)
simple_paths2 = []; counter = 0
for path in tqdm(nx.all_simple_paths(kg_orientation, source=s, target=t, cutoff=5)):
    simple_paths2 += [path]
    if counter == 25: break
    else: counter += 1

In [ ]:
# simple paths (i.e., no repeated nodes)
formats_path_information(kg=kg_orientation,
                         paths=simple_paths2,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

***
#### lisinopril (`CHEBI_6503`) and ingrowing toenails (`HP_0012710`)  <a class="anchor" id="path3"></a>


In [ ]:
s = obo.CHEBI_6503; t = obo.HP_0012710
try:
    kg_orientation = kg; ind = 'directed'
    spl_d = nx.shortest_path_length(kg, source=s, target=t)  
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))
    # get efficiency (measure of how efficiently information is exchanged between nodes)
    eff = nx.efficiency(undirected_kg, s, t)
    print('The efficiency between the nodes is: {}'.format(str(eff)))
except nx.NetworkXNoPath:
    print('There is no path between {} and {}'.format(str(s), str(t)))
    kg_orientation = undirected_kg; ind = 'undirected'
    spl_d = nx.shortest_path_length(undirected_kg, source=s, target=t)
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))

In [ ]:
shortest_paths4 = list(nx.all_shortest_paths(kg_orientation, s, t))
p_len = nx.shortest_path_length(kg_orientation, source=s, target=t)
v = 'is' if len(shortest_paths4) == 1 else 'are'
print('There {} {} shortest paths of length {}'.format(v, str(len(shortest_paths4)), str(p_len)))

In [ ]:
formats_path_information(kg=kg_orientation,
                         paths=shortest_paths4,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

*Simple Paths*

In [ ]:
# simple paths (i.e., no repeated nodes)
simple_paths3 = []; counter = 0
for path in tqdm(nx.all_simple_paths(kg_orientation, source=s, target=t, cutoff=5)):
    simple_paths3 += [path]
    if counter == 10: break
    else: counter += 1

In [ ]:
# simple paths (i.e., no repeated nodes)
formats_path_information(kg=kg_orientation,
                         paths=simple_paths3,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

***
#### lisinopril (`CHEBI_6503`) and  presbyopia (`MONDO_0001330`)  <a class="anchor" id="path4"></a>


In [ ]:
s = obo.CHEBI_6503; t = obo.MONDO_0001330
try:
    kg_orientation = kg; ind = 'directed'
    spl_d = nx.shortest_path_length(kg, source=s, target=t)  
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))
    # get efficiency (measure of how efficiently information is exchanged between nodes)
    eff = nx.efficiency(undirected_kg, s, t)
    print('The efficiency between the nodes is: {}'.format(str(eff)))
except nx.NetworkXNoPath:
    print('There is no path between {} and {}'.format(str(s), str(t)))
    kg_orientation = undirected_kg; ind = 'undirected'
    spl_d = nx.shortest_path_length(undirected_kg, source=s, target=t)
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))

In [ ]:
shortest_paths4 = list(nx.all_shortest_paths(kg_orientation, s, t))
p_len = nx.shortest_path_length(kg_orientation, source=s, target=t)
v = 'is' if len(shortest_paths4) == 1 else 'are'
print('There {} {} shortest paths of length {}'.format(v, str(len(shortest_paths4)), str(p_len)))

In [ ]:
formats_path_information(kg=kg_orientation,
                         paths=shortest_paths4,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

*Simple Paths*

In [ ]:
# simple paths (i.e., no repeated nodes)
simple_paths4 = []; counter = 0
for path in tqdm(nx.all_simple_paths(kg_orientation, source=s, target=t, cutoff=5)):
    simple_paths4 += [path]
    if counter == 25: break
    else: counter += 1

In [ ]:
# simple paths (i.e., no repeated nodes)
formats_path_information(kg=kg_orientation,
                         paths=simple_paths4,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

<br>

***

### Unknown Relationships

***
#### ivermectin (`CHEBI_6078`) and neurotoxicity (`MONDO_0005527`)  <a class="anchor" id="path5"></a>

When examining `ivermectin` and `neurotoxicity`. We observed that this pair had an efficiency of `1.0` and a single shortest path of length `1`.

In [ ]:
s = obo.CHEBI_6078; t = obo.MONDO_0005527
try:
    kg_orientation = kg; ind = 'directed'
    spl_d = nx.shortest_path_length(kg, source=s, target=t)  
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))
    # get efficiency (measure of how efficiently information is exchanged between nodes)
    eff = nx.efficiency(undirected_kg, s, t)
    print('The efficiency between the nodes is: {}'.format(str(eff)))
except nx.NetworkXNoPath:
    print('There is no path between {} and {}'.format(str(s), str(t)))
    kg_orientation = undirected_kg; ind = 'undirected'
    spl_d = nx.shortest_path_length(undirected_kg, source=s, target=t)
    print('The shortest path length is: {} ({})'.format(str(spl_d), ind))

In [ ]:
shortest_paths5 = list(nx.all_shortest_paths(kg_orientation, s, t))
p_len = nx.shortest_path_length(kg_orientation, source=s, target=t)
v = 'is' if len(shortest_paths5) == 1 else 'are'
print('There {} {} shortest paths of length {}'.format(v, str(len(shortest_paths5)), str(p_len)))

In [ ]:
formats_path_information(kg=kg_orientation,
                         paths=shortest_paths5,
                         path_type='shortest',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)

*Simple Paths*

In [ ]:
# simple paths (i.e., no repeated nodes)
simple_paths5 = []; counter = 0
for path in tqdm(nx.all_simple_paths(undirected_kg, source=s, target=t, cutoff=5)):
    simple_paths5 += [path]
    if counter == 25: break
    else: counter += 1

In [ ]:
# simple paths (i.e., no repeated nodes) -- forcing the kg to be undirected in order to get output
formats_path_information(kg=undirected_kg,
                         paths=simple_paths5,
                         path_type='simple',
                         metadata_func=metadata_formatter,
                         metadata_dict=None,
                         node_metadata=node_data_dict,
                         verbose=False,
                         rand=True,
                         sample_size=10)


<br>

***
***

This Notebook is part of the [**PheKnowLator Ecosystem**](https://zenodo.org/communities/pheknowlator-ecosystem/edit/)

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```

***